In [1]:
%matplotlib inline
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import uqer
from PyFin.api import *
from uqer import DataAPI as api
from QuantLib import *

plt.style.use('fivethirtyeight')
uqer.Client(token='f1b9bea1d0b4e489c5ab9b69c3e2326a1bee6057af858067dbd1546453f428b2')

16937@wmcloud.com 账号登录成功


In [3]:
start_date = '2017-04-01'
end_date = '2017-08-01'

dates = makeSchedule(start_date, end_date, '4m', 'china.sse', )

mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
contract = 'ru1709'

In [4]:
def get_contracts(contract, start_date, end_date):
    data = api.MktFutdGet(ticker=contract, beginDate=start_date, endDate=end_date, field='tradeDate,closePrice')
    data['tradeDate'] = pd.to_datetime(data.tradeDate)
    data.set_index('tradeDate', inplace=True)
    data['closePrice'] = data['closePrice'].astype(float)
    data['ret'] = data.closePrice.diff() / data.closePrice.shift(1)
    data['vol'] = data['ret'].rolling(window=22).std() * np.sqrt(249)
    return data

In [5]:
def get_contracts_bar(constract, start_date, end_date, unit=60):
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    
    datas = []
    while start_date < end_date:
        this_end_date = advanceDateByCalendar('china.sse', start_date, '10b')
        df = api.MktFutureBarHistDateRangeGet(instrumentID=contract,
                                              startDate=start_date.strftime('%Y%m%d'),
                                              endDate=this_end_date.strftime('%Y%m%d'),
                                              unit=unit,
                                              field='dataDate,barTime,closePrice')
        datas.append(df)
        start_date = this_end_date
    data = pd.concat(datas)
    data = data[(data.barTime >= '09:00') & (data.barTime <= '16:00')]
    data['tradeDate'] = pd.to_datetime(data.dataDate + 'T' +  data.barTime)
    data['closePrice'] = data['closePrice'].astype(float)
    data['ret'] = data.closePrice.diff() / data.closePrice.shift(1)
    data['vol'] = data['ret'].rolling(window=int(22 * 4 * 60 / unit)).std() * np.sqrt(249) * np.sqrt(4 * 60 / unit)
    data.set_index('tradeDate', inplace=True)
    return data[['closePrice', 'ret', 'vol']]

In [6]:
mkt_data = get_contracts_bar(contract, mkt_start_date, end_date)

In [7]:
strangle_width = 400.
lower_protect_area = 1000.
upper_protect_area = 800.
volatility = 0.42
risk_free_rate = 0.
dividend_rate = 0.

In [10]:
dates

[datetime.datetime(2017, 4, 5, 0, 0), datetime.datetime(2017, 8, 1, 0, 0)]

# Dynamical Hedging
------------

In [128]:
def cal_naive_delta(spot, pp1, pp2, cp1, cp2):
    
    if spot <= pp2.strike():
        return 0.
    elif spot <= pp1.strike():
        return -1.
    elif spot <= cp1.strike():
        return 0.
    elif spot <= cp2.strike():
        return 1.
    else:
        return 0.
    
def composite_payoff(spot, pp1, pp2, cp1, cp2):
    return pp1(spot) - pp2(spot) + cp1(spot) - cp2(spot)

def hedging_contract(start_date, maturity_date, mkt_data, premium=None, naive_hedge=False, american_exercise=False):
    spot_price = mkt_data[mkt_data.index < start_date.ISO()]['closePrice']
    
    if hasattr(spot_price, '__iter__'):
        spot_price = spot_price[-1]
    
    lower_strike = spot_price
    lower_bound = lower_strike - lower_protect_area
    upper_strike = lower_strike + strangle_width
    upper_bound = upper_strike + upper_protect_area
    
    day_count = Actual365Fixed()
    calendar = China(China.SSE)
    
    calculation_date = start_date
    Settings.instance().evaluationDate = calculation_date

    spot_handle = RelinkableQuoteHandle(SimpleQuote(spot_price))
    vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

    flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
    dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
    flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

    bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
    
    put_payoff1 = PlainVanillaPayoff(Option.Put, lower_strike)
    put_payoff2 = PlainVanillaPayoff(Option.Put, lower_bound)
    call_payoff1 = PlainVanillaPayoff(Option.Call, upper_strike)
    call_payoff2 = PlainVanillaPayoff(Option.Call, upper_bound)

    exercise = EuropeanExercise(maturity_date)
    put_option1 = VanillaOption(put_payoff1, exercise)
    put_option2 = VanillaOption(put_payoff2, exercise)
    call_option1 = VanillaOption(call_payoff1, exercise)
    call_option2 = VanillaOption(call_payoff2, exercise)

    engine = AnalyticEuropeanEngine(bsm_process)
    put_option1.setPricingEngine(engine)
    put_option2.setPricingEngine(engine)
    call_option1.setPricingEngine(engine)
    call_option2.setPricingEngine(engine)
    
    if not premium:
        premium = put_option1.NPV() - put_option2.NPV() + call_option1.NPV() - call_option2.NPV()
    
    price_series = mkt_data.loc[start_date.ISO(): maturity_date.ISO()]
    previous_delta = 0.
    previous_price = 0.
    trading_pnl = 0.
    trading_cost = 0.
    for p in price_series.iterrows():
        date = Date.from_date(p[0])
        price = p[1]['closePrice']
        vol = p[1]['vol']
        if date != maturity_date:
            Settings.instance().evaluationDate = date
        spot_handle.linkTo(SimpleQuote(price))
        vol_handle.linkTo(SimpleQuote(vol))
        bsm_delta = put_option1.delta() - put_option2.delta() + call_option1.delta() - call_option2.delta()
        
        if naive_hedge:
            delta = cal_naive_delta(price, put_payoff1, put_payoff2, call_payoff1, call_payoff2)
        else:
            delta = bsm_delta
        
        trading_pnl += (price - previous_price) * previous_delta
        trading_cost += np.abs(delta - previous_delta) * 0.5 * (price + previous_price) * 0.0015     
        if american_exercise:
            payoff = composite_payoff(price,
                                      put_payoff1,
                                      put_payoff2,
                                      call_payoff1,
                                      call_payoff2)
            
            if price >= call_payoff2.strike() - 25.0 and payoff >= upper_protect_area - 25.:
                break
            elif price <= put_payoff2.strike() + 25.0 and payoff >= upper_protect_area - 25.:
                break
        
        previous_price = price
        previous_delta = delta
        
    final_price = mkt_data.loc[date.ISO()]['closePrice']
    if hasattr(final_price, '__iter__'):
        final_price = final_price[-1]
    final_payoff = composite_payoff(final_price,
                                    put_payoff1,
                                    put_payoff2,
                                    call_payoff1,
                                    call_payoff2)
    
    pnl_final = -final_payoff + premium + trading_pnl - trading_cost
    
    return spot_price, final_price, vol, final_payoff, premium, trading_pnl, pnl_final, trading_cost

In [129]:
maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[1], '-1b'))
start_date = Date.from_date(dates[0])

hedging_contract(start_date, maturity_date, mkt_data, 646.)

(16335.0,
 12545.0,
 0.40250647967576453,
 1000.0,
 646.0,
 171.77765850015768,
 -186.82633302507787,
 4.60399152523556)

# Historical Simulation
-----------------

## 日间对冲
------------

In [130]:
strangle_width = 400.
lower_protect_area = 1000.
upper_protect_area = 800.
volatility = 0.42
risk_free_rate = 0.
dividend_rate = 0.
premium = 769.

In [228]:
contract_definition = {
    'ru0509': {
        'start_date': '2005-04-01',
        'end_date': '2005-08-01'
    },
    'ru0609': {
        'start_date': '2006-04-01',
        'end_date': '2006-08-01'
    },
    'ru0709': {
        'start_date': '2007-04-01',
        'end_date': '2007-08-01'
    },
    'ru0809': {
        'start_date': '2008-04-01',
        'end_date': '2008-08-01'
    },
    'ru0909': {
        'start_date': '2009-04-01',
        'end_date': '2009-08-01'
    },
    'ru1009': {
        'start_date': '2010-04-01',
        'end_date': '2010-08-01'
    },
    'ru1109': {
        'start_date': '2011-04-01',
        'end_date': '2011-08-01'
    },
    'ru1209': {
        'start_date': '2012-04-01',
        'end_date': '2012-08-01'
    },
    'ru1309': {
        'start_date': '2013-04-01',
        'end_date': '2013-08-01'
    },
    'ru1409': {
        'start_date': '2014-04-01',
        'end_date': '2014-08-01'
    },
    'ru1509': {
        'start_date': '2015-04-01',
        'end_date': '2015-08-01'
    },
    'ru1609': {
        'start_date': '2016-04-01',
        'end_date': '2016-08-01'
    },
    'ru1709': {
        'start_date': '2017-04-01',
        'end_date': '2017-08-01'
    },
}

In [229]:
%%time

cols = ['合约', '到期日', '初始价', '收盘价', '实现波动率', '赔付', '期权费', '交易损益', '对冲损益' , '对冲交易成本', 'delta 1 对冲损益', 'delta 1 对冲交易成本']
df = pd.DataFrame(columns=cols)

for contract in contract_definition:
    start_date = contract_definition[contract]['start_date']
    end_date = contract_definition[contract]['end_date']
    dates = makeSchedule(start_date, end_date, '4m', 'china.sse')
    mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
    mkt_data = get_contracts(contract, mkt_start_date, end_date)
    
    for i, s_date in enumerate(dates[:-1]):
        maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[i+1], '-1b'))
        start_date = Date.from_date(s_date)

        hedge = hedging_contract(start_date, maturity_date, mkt_data, premium)
        delta1_hedge = hedging_contract(start_date, maturity_date, mkt_data, premium, True)
        py_dt = dt.datetime(maturity_date.year(), maturity_date.month(), maturity_date.dayOfMonth())
        df = df.append(dict(zip(cols, (contract, py_dt) + hedge + delta1_hedge[-2:])), ignore_index=True)

Wall time: 1.05 s


In [230]:
df['不对冲损益'] = -df['赔付'] + df['期权费']

In [231]:
df

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益
0,ru0509,2005-07-29,13410.0,16640.0,0.219884,800.0,769.0,74.472263,22.061139,21.411125,44.31625,124.68375,-31.0
1,ru0609,2006-07-31,20190.0,23740.0,0.374284,800.0,769.0,-68.498223,-106.915665,7.417442,-306.56125,185.56125,-31.0
2,ru0709,2007-07-31,22480.0,19160.0,0.193958,1000.0,769.0,63.437664,-178.222058,10.659722,2160.40125,338.59875,-231.0
3,ru0809,2008-07-31,22805.0,28035.0,0.257334,800.0,769.0,-124.217880,-163.600447,8.382567,1173.94875,260.05125,-31.0
4,ru0909,2009-07-31,13550.0,16870.0,0.257517,800.0,769.0,5.000685,-35.023759,9.024443,2162.17875,381.82125,-31.0
5,ru1009,2010-07-30,24515.0,23680.0,0.183390,835.0,769.0,-9.098860,-83.007430,7.908570,-772.68500,326.68500,-66.0
6,ru1109,2011-07-29,34895.0,34445.0,0.194590,450.0,769.0,19.013578,332.629217,5.384361,-891.60875,655.60875,319.0
7,ru1209,2012-07-31,27120.0,22670.0,0.259026,1000.0,769.0,202.585781,-37.239199,8.824980,318.82875,320.17125,-231.0
8,ru1309,2013-07-31,22165.0,16865.0,0.295579,1000.0,769.0,183.842293,-52.072991,4.915284,826.62625,47.37375,-231.0
9,ru1409,2014-07-31,15745.0,14210.0,0.185704,1000.0,769.0,71.196773,-172.264951,12.461724,1329.56250,89.43750,-231.0


In [232]:
print("欧式行权: 2005 - 2017")
df.groupby(df['到期日'].dt.year).mean()

欧式行权: 2005 - 2017


,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益
到期日,,,,,,,,,,,
2005,13410.0,16640.0,0.219884,800.0,769.0,74.472263,22.061139,21.411125,44.31625,124.68375,-31.0
2006,20190.0,23740.0,0.374284,800.0,769.0,-68.498223,-106.915665,7.417442,-306.56125,185.56125,-31.0
2007,22480.0,19160.0,0.193958,1000.0,769.0,63.437664,-178.222058,10.659722,2160.40125,338.59875,-231.0
2008,22805.0,28035.0,0.257334,800.0,769.0,-124.217880,-163.600447,8.382567,1173.94875,260.05125,-31.0
2009,13550.0,16870.0,0.257517,800.0,769.0,5.000685,-35.023759,9.024443,2162.17875,381.82125,-31.0
2010,24515.0,23680.0,0.183390,835.0,769.0,-9.098860,-83.007430,7.908570,-772.68500,326.68500,-66.0
2011,34895.0,34445.0,0.194590,450.0,769.0,19.013578,332.629217,5.384361,-891.60875,655.60875,319.0
2012,27120.0,22670.0,0.259026,1000.0,769.0,202.585781,-37.239199,8.824980,318.82875,320.17125,-231.0
2013,22165.0,16865.0,0.295579,1000.0,769.0,183.842293,-52.072991,4.915284,826.62625,47.37375,-231.0


In [233]:
df.std()

初始价               6817.506382
收盘价               6899.241620
实现波动率                0.075995
赔付                 196.326031
期权费                  0.000000
交易损益               106.688566
对冲损益               148.317987
对冲交易成本               4.580873
delta 1 对冲损益      1147.581080
delta 1 对冲交易成本     162.893391
不对冲损益              196.326031
dtype: float64

In [234]:
df.mean()

初始价               19778.461538
收盘价               19361.538462
实现波动率                 0.267762
赔付                  811.923077
期权费                 769.000000
交易损益                 33.599113
对冲损益                -19.154519
对冲交易成本                9.830555
delta 1 对冲损益        338.714135
delta 1 对冲交易成本      261.055096
不对冲损益               -42.923077
dtype: float64

## 日内对冲
---------------

In [235]:
%%time

cols = ['合约', '到期日', '初始价', '收盘价', '实现波动率', '赔付', '期权费', '交易损益', '对冲损益' , '对冲交易成本', 'delta 1 对冲损益', 'delta 1 对冲交易成本']
df2 = pd.DataFrame(columns=cols)

for contract in list(contract_definition.keys())[5:]:
    start_date = contract_definition[contract]['start_date']
    end_date = contract_definition[contract]['end_date']
    dates = makeSchedule(start_date, end_date, '4m', 'china.sse')
    mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
    mkt_data = get_contracts_bar(contract, mkt_start_date, end_date, unit=60)
    
    for i, s_date in enumerate(dates[:-1]):
        maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[i+1], '-1b'))
        start_date = Date.from_date(s_date)

        hedge = hedging_contract(start_date, maturity_date, mkt_data, premium)
        delta1_hedge = hedging_contract(start_date, maturity_date, mkt_data, premium, True)
        py_dt = dt.datetime(maturity_date.year(), maturity_date.month(), maturity_date.dayOfMonth())
        df2 = df2.append(dict(zip(cols, (contract, py_dt) + hedge + delta1_hedge[-2:])), ignore_index=True)

Wall time: 15.1 s


In [246]:
df2['不对冲损益'] = -df2['赔付'] + df2['期权费']

In [247]:
df2

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益
0,ru1009,2010-07-30,24515.0,23680.0,0.147151,835.0,769.0,-44.789349,-128.502136,17.712787,275.59500,993.40500,-66.0
1,ru1109,2011-07-29,34895.0,34445.0,0.207916,450.0,769.0,-17.495701,288.483164,13.021136,-3979.92875,2348.92875,319.0
2,ru1209,2012-07-31,27120.0,22670.0,0.203450,1000.0,769.0,216.825831,-35.665634,21.491465,-1864.38250,948.38250,-231.0
3,ru1309,2013-07-31,22165.0,16865.0,0.247096,1000.0,769.0,218.973881,-25.433285,13.407165,1017.35500,111.64500,-231.0
4,ru1409,2014-07-31,15775.0,14210.0,0.171678,1000.0,769.0,80.453324,-170.227155,19.680479,887.46750,261.53250,-231.0
5,ru1509,2015-07-31,12540.0,11995.0,0.332727,545.0,769.0,-77.703231,122.488982,23.807787,441.87875,337.12125,224.0
6,ru1609,2016-07-29,11370.0,10845.0,0.393223,525.0,769.0,-80.511020,140.683995,22.804986,-834.79375,743.79375,244.0
7,ru1709,2017-07-31,16335.0,12545.0,0.316898,1000.0,769.0,151.356154,-88.206045,8.562199,-328.10500,147.10500,-231.0


In [248]:
df2.groupby(df2['到期日'].dt.year).mean()

,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益
到期日,,,,,,,,,,,
2010,24515.0,23680.0,0.147151,835.0,769.0,-44.789349,-128.502136,17.712787,275.59500,993.40500,-66.0
2011,34895.0,34445.0,0.207916,450.0,769.0,-17.495701,288.483164,13.021136,-3979.92875,2348.92875,319.0
2012,27120.0,22670.0,0.203450,1000.0,769.0,216.825831,-35.665634,21.491465,-1864.38250,948.38250,-231.0
2013,22165.0,16865.0,0.247096,1000.0,769.0,218.973881,-25.433285,13.407165,1017.35500,111.64500,-231.0
2014,15775.0,14210.0,0.171678,1000.0,769.0,80.453324,-170.227155,19.680479,887.46750,261.53250,-231.0
2015,12540.0,11995.0,0.332727,545.0,769.0,-77.703231,122.488982,23.807787,441.87875,337.12125,224.0
2016,11370.0,10845.0,0.393223,525.0,769.0,-80.511020,140.683995,22.804986,-834.79375,743.79375,244.0
2017,16335.0,12545.0,0.316898,1000.0,769.0,151.356154,-88.206045,8.562199,-328.10500,147.10500,-231.0


In [249]:
df2.std()

初始价               8073.416005
收盘价               8069.770282
实现波动率                0.086553
赔付                 246.148003
期权费                  0.000000
交易损益               127.720688
对冲损益               156.691309
对冲交易成本               5.415964
delta 1 对冲损益      1682.289005
delta 1 对冲交易成本     740.058086
不对冲损益              246.148003
dtype: float64

In [250]:
df2.mean()

初始价               20589.375000
收盘价               18406.875000
实现波动率                 0.252517
赔付                  794.375000
期权费                 769.000000
交易损益                 55.888736
对冲损益                 12.952736
对冲交易成本               17.561000
delta 1 对冲损益       -548.114219
delta 1 对冲交易成本      736.489219
不对冲损益               -25.375000
dtype: float64

## 比较
-------------

In [251]:
df = df[df['合约'].isin(df2['合约'])].reset_index(drop=True)

In [252]:
s = df['不对冲损益'] - df2['不对冲损益']

In [253]:
df['diff'] = s

In [254]:
df[df['diff'] != 0]

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益,diff


In [255]:
df2[df['diff'] != 0]

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益


## 日间对冲美式
-----------

In [262]:
%%time

cols = ['合约', '到期日', '初始价', '收盘价', '实现波动率', '赔付', '期权费', '交易损益', '对冲损益' , '对冲交易成本', 'delta 1 对冲损益', 'delta 1 对冲交易成本']
df = pd.DataFrame(columns=cols)

for contract in contract_definition:
    start_date = contract_definition[contract]['start_date']
    end_date = contract_definition[contract]['end_date']
    dates = makeSchedule(start_date, end_date, '4m', 'china.sse')
    mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
    mkt_data = get_contracts(contract, mkt_start_date, end_date)
    
    for i, s_date in enumerate(dates[:-1]):
        maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[i+1], '-1b'))
        start_date = Date.from_date(s_date)

        hedge = hedging_contract(start_date, maturity_date, mkt_data, premium, american_exercise=True)
        delta1_hedge = hedging_contract(start_date, maturity_date, mkt_data, premium, True, american_exercise=True)
        py_dt = dt.datetime(maturity_date.year(), maturity_date.month(), maturity_date.dayOfMonth())
        df = df.append(dict(zip(cols, (contract, py_dt) + hedge + delta1_hedge[-2:])), ignore_index=True)

Wall time: 898 ms


In [263]:
df['不对冲损益'] = -df['赔付'] + df['期权费']

In [264]:
df

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益
0,ru0509,2005-07-29,13410.0,14740.0,0.174206,800.0,769.0,-89.310735,-138.360325,18.049590,44.31625,124.68375,-31.0
1,ru0609,2006-07-31,20190.0,21790.0,0.233272,800.0,769.0,-39.082031,-72.531098,2.449068,-306.56125,185.56125,-31.0
2,ru0709,2007-07-31,22480.0,24190.0,0.177264,800.0,769.0,-46.758794,-82.825918,5.067124,731.59250,137.40750,-31.0
3,ru0809,2008-07-31,22805.0,21700.0,0.244178,1000.0,769.0,17.697762,-214.134284,0.832046,174.47375,49.52625,-231.0
4,ru0909,2009-07-31,13550.0,15005.0,0.416781,800.0,769.0,-40.050926,-71.909282,0.858356,155.47750,73.52250,-31.0
5,ru1009,2010-07-30,24515.0,23500.0,0.230581,1000.0,769.0,24.226333,-209.232191,2.458524,-1077.13000,221.13000,-231.0
6,ru1109,2011-07-29,34895.0,36680.0,0.402120,800.0,769.0,-22.831801,-54.326231,0.494430,-782.19125,186.19125,-31.0
7,ru1209,2012-07-31,27120.0,26100.0,0.183334,1000.0,769.0,20.922280,-215.049297,4.971576,-288.10375,242.10375,-231.0
8,ru1309,2013-07-31,22165.0,20355.0,0.270112,1000.0,769.0,42.010562,-190.263221,1.273783,826.62625,47.37375,-231.0
9,ru1409,2014-07-31,15745.0,14200.0,0.246783,1000.0,769.0,98.887960,-134.874963,2.762923,1053.76375,45.23625,-231.0


In [265]:
print("近似美式行权: 2013 - 2017")
df.groupby(df['到期日'].dt.year).mean()

近似美式行权: 2013 - 2017


,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,对冲交易成本,delta 1 对冲损益,delta 1 对冲交易成本,不对冲损益
到期日,,,,,,,,,,,
2005,13410.0,14740.0,0.174206,800.0,769.0,-89.310735,-138.360325,18.049590,44.31625,124.68375,-31.0
2006,20190.0,21790.0,0.233272,800.0,769.0,-39.082031,-72.531098,2.449068,-306.56125,185.56125,-31.0
2007,22480.0,24190.0,0.177264,800.0,769.0,-46.758794,-82.825918,5.067124,731.59250,137.40750,-31.0
2008,22805.0,21700.0,0.244178,1000.0,769.0,17.697762,-214.134284,0.832046,174.47375,49.52625,-231.0
2009,13550.0,15005.0,0.416781,800.0,769.0,-40.050926,-71.909282,0.858356,155.47750,73.52250,-31.0
2010,24515.0,23500.0,0.230581,1000.0,769.0,24.226333,-209.232191,2.458524,-1077.13000,221.13000,-231.0
2011,34895.0,36680.0,0.402120,800.0,769.0,-22.831801,-54.326231,0.494430,-782.19125,186.19125,-31.0
2012,27120.0,26100.0,0.183334,1000.0,769.0,20.922280,-215.049297,4.971576,-288.10375,242.10375,-231.0
2013,22165.0,20355.0,0.270112,1000.0,769.0,42.010562,-190.263221,1.273783,826.62625,47.37375,-231.0


In [266]:
df.std()

初始价               6817.506382
收盘价               6796.235360
实现波动率                0.085509
赔付                 105.393037
期权费                  0.000000
交易损益                52.221303
对冲损益                65.096920
对冲交易成本               4.633170
delta 1 对冲损益       654.677411
delta 1 对冲交易成本      73.760126
不对冲损益              105.393037
dtype: float64

In [267]:
df.mean()

初始价               19778.461538
收盘价               19982.692308
实现波动率                 0.272050
赔付                  890.769231
期权费                 769.000000
交易损益                 -3.620287
对冲损益               -128.967401
对冲交易成本                3.577884
delta 1 对冲损益         48.431731
delta 1 对冲交易成本      114.799038
不对冲损益              -121.769231
dtype: float64